In [14]:
import gzip
import numpy as np
import struct
import matplotlib.pyplot as plt
from status import Status, parse_status
from refnx.reduce.event import events

In [15]:
def event_sef(buf):
    # get a list of events from the SEF based on a binary buffer
    it = struct.iter_unpack(">lQ2s", buf)
    return list(it)
    
def predicted_frame(ev):
    # figures out which frame in the SEF corresponds to the first frame
    # in the NEF
    frames =  [ev[i][0] for i in range(len(ev)) if ev[i][-1] == b'\x00~']
    t0 = [ev[i][1] for i in range(len(ev)) if ev[i][-1] == b'\x00~']
    t4 = [ev[i][1] for i in range(len(ev)) if ev[i][-1] != b'\x00~']
    frames4 = [ev[i][0] for i in range(len(ev)) if ev[i][-1] == b'\x00D']

    idx = np.searchsorted(np.array(t0) / 1e9, dataset_start_time_t)
    print(f"array search: {int(idx)}, nearest time: {np.argmin(((np.array(t0)) / 1e9 - dataset_start_time_t)**2)}")
    return int(idx), np.squeeze(t0), np.squeeze(t4)

def frame_with_first_neutrons(ev, period=4):
    # predict the first frame in the NEF to have neutrons, if one of the
    # choppers is running at a geared speed. This is predicted from the SEF
    idx, t0, t4 = predicted_frame(ev)
    
    frames4 = [ev[i][0] for i in range(len(ev)) if ev[i][-1] == b'\x00D'][0]
    perfect_4 = np.arange(frames4, idx + period + 5, period)
    actual_4_frame = np.searchsorted(perfect_4, idx)
    return int(perfect_4[actual_4_frame] - idx)

In [17]:
LOC = 'DAQ_2025-06-19T11-36-57/'

In [18]:
s = Status("manager")
status = parse_status(s.from_file(LOC))[0]
dataset_start_time_t = status['dataset_start_time_t']

with gzip.GzipFile(f'{LOC}/DATA_0/EOS.gz', 'rb') as f:
    buf = f.read(1_000_000_000)

In [22]:
ev = event_sef(buf)
idx, t0, t4 = predicted_frame(ev)
fwfn = frame_with_first_neutrons(ev)
print(fwfn)

array search: 48, nearest time: 48
array search: 48, nearest time: 48
3


In [25]:
nef = events(f"{LOC}DATA_0/EOS.bin")
frames = nef[0][0]
_idx = np.arange(max(frames) + 1)
neut_per_frame = np.bincount(frames)
coincident = _idx[neut_per_frame > 10]
print(coincident[:50])
print(neut_per_frame[:50])
assert coincident[0] == fwfn

[  3   7  11  15  19  23  27  31  35  39  43  47  51  55  59  63  67  71
  75  79  83  87  91  95  99 103 107 111 115 119 123 127 131 135 139 143
 147 151 155 159 163 167 171 175 179 183 187 191 195 199]
[ 0  0  1 73  0  0  1 89  0  0  0 84  2  0  0 74  1  1  0 64  1  1  0 74
  0  0  0 64  0  0  2 79  1  1  0 82  1  0  0 82  0  0  0 63  0  0  0 75
  0  0]


In [21]:
_t0 = np.array(t0) / 1e9
_t0[idx - 2], _t0[idx - 1], _t0[idx], _t0[idx + 1], _t0[idx + 2], dataset_start_time_t

(np.float64(1750297019.9464233),
 np.float64(1750297019.9879086),
 np.float64(1750297020.0294693),
 np.float64(1750297020.071486),
 np.float64(1750297020.1130157),
 1750297020.023683)

In [12]:
1/np.mean(np.diff(_t0)), 1/np.mean(np.diff(t4)/1e9)

(np.float64(24.000049091747677), np.float64(4.4317284259216505))

In [13]:
# Cell calculates the time lag of the slower chopper compared to first
# epoch times for c1 that has c4 as well
c1_with_4 = np.array(t0)[np.array(frames4)]
np.mean((np.array(t4) - c1_with_4)/1e6), np.std((np.array(t4) - c1_with_4)/1e6)

# (np.float64(6.5960680104166665), np.float64(0.1701891744150397))
# (np.float64(6.7056619444444445), np.float64(0.7798491055443618))

NameError: name 'frames4' is not defined